# Part 2a: CNNs (LFW Dataset)

In [1]:
import time

import numpy as np
import torch, torchvision

print("PyTorch Version :\t", torch.__version__)
print("CUDA Available? :\t", torch.cuda.is_available())

PyTorch Version :	 2.0.1+cu118
CUDA Available? :	 True


### Data Preparation

In [2]:
from sklearn.datasets import fetch_lfw_people

# Cached in "~/scikit_learn_data" after first download
lfw_people = fetch_lfw_people(min_faces_per_person=70, resize=0.4)

X = lfw_people.images
y = lfw_people.target

In [3]:
from sklearn.model_selection import train_test_split

# Split into a training set and a test set using a stratified k fold
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, 
    random_state=42)

# Normalise and reshape data into 4D array (added dimension is channels)
X_train = X_train[:, np.newaxis, :, :] / 255
X_test = X_test[:, np.newaxis, :] / 255

# Load all data and labels into Tensor objects
X_train = torch.tensor(X_train, device="cuda")
y_train = torch.tensor(y_train, device="cuda")
X_test = torch.tensor(X_test, device="cuda")
y_test = torch.tensor(y_test, device="cuda")

### Create a CNN Model

In [18]:
import torch.nn as nn
import torch.nn.functional as F

class LFWCNN(nn.Module):
    def __init__(self, in_channels, num_classes):
        super().__init__()
        self.in_channels = in_channels
        self.num_classes = num_classes
        self.sequential = self._make_conv(self.in_channels)
        self.classifier = self._make_clsf(self.num_classes)

    def forward(self, x):
        x = self.sequential(x)
        x = torch.flatten(x, 1)
        x = self.classifier(x)
        return x

    def _make_conv(self, in_channels):
        layers = [
            nn.Conv2d(in_channels, 32, kernel_size=3, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(inplace=True),
            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
        ]
        return nn.Sequential(*layers)
    
    def _make_clsf(self, out_channels):
        layers = [
            nn.Linear(64 * 50 * 37, 256),
            nn.Linear(256, out_channels)
        ]
        return nn.Sequential(*layers)

In [24]:
# Define the model
model = LFWCNN(1, num_classes=len(np.unique(y)))
model.to("cuda")

# Model info
print("Model No. of Parameters:", 
    sum([param.nelement() for param in model.parameters()]))
print(model)

# Loss function and optimizer + hyperparameters
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

Model No. of Parameters: 30331463
LFWCNN(
  (sequential): Sequential(
    (0): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace=True)
  )
  (classifier): Sequential(
    (0): Linear(in_features=118400, out_features=256, bias=True)
    (1): Linear(in_features=256, out_features=7, bias=True)
  )
)


### Train the CNN Model

In [25]:
NUM_EPOCHS = 20
BATCH_SIZE = 64

# Split the training further into training and validation sets
# X_train, X_val = X_train[:-64], X_train[-64:]
# y_train, y_val = y_train[:-64], y_train[-64:]

# Batch the training data
X_train_batches = torch.split(X_train, BATCH_SIZE)
y_train_batches = torch.split(y_train, BATCH_SIZE)

# Set the model in training mode
model.train()
start = time.time()
for epoch in range(NUM_EPOCHS):
    for i, (images, labels) in enumerate(zip(X_train_batches, y_train_batches)):

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    # # Evaluate using validation set
    # with torch.no_grad():
    #     outputs = model(X_val)
    #     _, predicted = torch.max(outputs.data, 1)
    #     total = y_val.size(0)
    #     correct = (predicted == y_val).sum().item()
    #     val_acc = (correct / total) * 100

    # Training report
    print(f"Epoch [{epoch+1}/{NUM_EPOCHS}]\tLoss: {loss.item():.5f}", end="")
    # print(f"\tVal.: {val_acc:.2f}%", end="")
    print()

end = time.time()
elapsed = end - start
print(f"Total training time: {elapsed} s")

Epoch [1/20]	Loss: 22.76771
Epoch [2/20]	Loss: 3.71115
Epoch [3/20]	Loss: 0.00000
Epoch [4/20]	Loss: 0.00000
Epoch [5/20]	Loss: 0.00000
Epoch [6/20]	Loss: 0.00000
Epoch [7/20]	Loss: 0.00000
Epoch [8/20]	Loss: 0.00000
Epoch [9/20]	Loss: 0.00000
Epoch [10/20]	Loss: 0.00000
Epoch [11/20]	Loss: 0.00000
Epoch [12/20]	Loss: 0.00000
Epoch [13/20]	Loss: 0.00000
Epoch [14/20]	Loss: 0.00000
Epoch [15/20]	Loss: 0.00000
Epoch [16/20]	Loss: 0.00000
Epoch [17/20]	Loss: 0.00000
Epoch [18/20]	Loss: 0.00000
Epoch [19/20]	Loss: 0.00000
Epoch [20/20]	Loss: 0.00000
Total training time: 29.29731297492981 s


### Test the CNN Model

In [26]:
# Split the test data into batches
X_test_batches = torch.split(X_test, BATCH_SIZE)
y_test_batches = torch.split(y_test, BATCH_SIZE)

# Set the model in evaluation (test) mode
model.eval()
start = time.time()
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in zip(X_test_batches, y_test_batches):
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
end = time.time()
elapsed = end - start
print(f"Test accuracy: {100 * correct / total:.2f}% (in {elapsed:.3f}s)")

Test accuracy: 80.75% (in 0.173s)
